In [13]:
import pandas as pd
import xmltodict
import requests
import os


def get_url_index(url):
    response = requests.get(url).content.decode('utf8')
    xml = xmltodict.parse(response)

    last_url = [l['@href'] for l in xml['feed']['link'] if (l['@rel'] == 'last')][0]
    last_index = [int(s.replace('page=', '')) for s in last_url.split('/') if ('page=' in s)][0]

    return last_index

def appstore_crawler(appid, outfile='./appstore_reviews.csv'):
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=%i/sortby=mostrecent/xml' % appid
    try:
        last_index = get_url_index(url)
    except Exception as e:
        print (url)
        print ('\tNo Reviews: appid %i' %appid)
        print ('\tException:', e)
        return

    result = list()
    for idx in range(1, last_index+1):
        url = "https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=%i/sortby=mostrecent/xml?urlDesc=/customerreviews/id=%i/sortBy=mostRecent/xml" % (idx, appid, appid)
        print(url)

        response = requests.get(url).content.decode('utf8')
        try:
            xml = xmltodict.parse(response)
        except Exception as e:
            print ('\tXml Parse Error %s\n\tSkip %s :' %(e, url))
            continue

        try:
            num_reivews= len(xml['feed']['entry'])
        except Exception as e:
            print ('\tNo Entry', e)
            continue

        try:
            xml['feed']['entry'][0]['author']['name']
            single_reviews = False
        except:
            #print ('\tOnly 1 review!!!')
            single_reviews = True
            pass

        if single_reviews:
                result.append({
                    'USER': xml['feed']['entry']['author']['name'],
                    'DATE': xml['feed']['entry']['updated'],
                    'STAR': int(xml['feed']['entry']['im:rating']),
                    'LIKE': int(xml['feed']['entry']['im:voteSum']),
                    'TITLE': xml['feed']['entry']['title'],
                    'REVIEW': xml['feed']['entry']['content'][0]['#text'],
                })
        else:
            for i in range(len(xml['feed']['entry'])):
                result.append({
                    'USER': xml['feed']['entry'][i]['author']['name'],
                    'DATE': xml['feed']['entry'][i]['updated'],
                    'STAR': int(xml['feed']['entry'][i]['im:rating']),
                    'LIKE': int(xml['feed']['entry'][i]['im:voteSum']),
                    'TITLE': xml['feed']['entry'][i]['title'],
                    'REVIEW': xml['feed']['entry'][i]['content'][0]['#text'],
                })

    res_df = pd.DataFrame(result)
    res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S")
    res_df.to_csv(outfile, encoding='utf-8-sig', index=False)
    print ('Save reviews to file: %s \n' %(outfile))


if __name__ == '__main__':

    app_id = 1622149150
    app_id2 = 1472679606
    app_id3 = 1622144454
    app_id4 =1480466201
#     outfile = os.path.join('appstore_' + str(app_id)+'.csv')
#     outfile2 = os.path.join("appstore2_" + str(app_id2) + '.csv')
    outfile3 = os.path.join("Grandstage" + str(app_id3) + '.csv')
    outfile4 = os.path.join("OTS" + str(app_id4) + '.csv')
    appstore_crawler(app_id, outfile=outfile)
    appstore_crawler(app_id2, outfile=outfile2)
    appstore_crawler(app_id3, outfile=outfile3)
    appstore_crawler(app_id4, outfile=outfile4)

https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1622149150/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1622149150/sortBy=mostRecent/xml
Save reviews to file: appstore_1622149150.csv 

https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1472679606/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1472679606/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=2/id=1472679606/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1472679606/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=3/id=1472679606/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1472679606/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=4/id=1472679606/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1472679606/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=5/id=1472679606/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1472679606/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customer

In [20]:
import pandas as pd
a = pd.read_csv("ABC-MART.csv")
b = pd.read_csv("A-RT.csv")
c = pd.read_csv("Grandstage.csv")
d = pd.read_csv("OTS.csv")

total = pd.concat([a,b,c,d], axis=0)
total.to_csv("total_abc_revies.csv")

In [21]:
total

,USER,DATE,STAR,LIKE,TITLE,REVIEW
0,dddd.d,2023-02-06 18:35:42-07:00,1,0,개느림 진짜 미치게느림,처음에 내 폰이 문젠줄알고 개당황했는데 앱자체가 미치게느려서 그냥 아무것도 할수가없...
1,ㄴㅇㅏ,2023-02-01 20:13:29-07:00,1,0,쿠폰을 줘도 못씀,쿠폰 준다고 앱 깔아서 카카오톡 친구 추가까지 했는데 일부 상품 제외라면서 할인 안...
2,LeoL🦁🦁,2023-01-20 17:12:46-07:00,1,0,ㄱㅂㅅ,진짜 최악이네요 주문한 상품의 결제가 어느순간 보니 취소되어있지 않나 고객센터는 전...
3,엑릭릭릭,2023-01-14 23:03:18-07:00,1,0,별점도 아까운 어플임,ABC마트가 망한다면 90%는 이 어플때문임\n반응속도부터 물건 찾는것도 ㅈㄹ 같고...
4,".,?..ㄷ",2023-01-11 17:45:23-07:00,1,0,최악의 쓰레기 어플,그냥 속도만 보면 답답해서 회원탈퇴하고싶게 만드는 앱\n걍 탈퇴함
...,...,...,...,...,...,...
4,옹옹옹오오,2020-03-08 18:11:07-07:00,1,0,앱 왜 만드신건가요..,로그인하면 오류 뜨고.. 웹기반이면 어플 없애고 웹으로만 하세요...
5,심시티의 어느심,2020-02-23 02:23:01-07:00,1,0,어플 개판이네,반응속도도 구리고 사이즈도 안뜨고 쿠폰 적용상품인데 쿠폰적용도안됨
6,얌채ㅋ,2020-02-21 01:49:03-07:00,1,0,어플좀 신경 써서 만드세요,인터페이스가 별로고 반응속도도 느려서 영 못쓰겠네요 그리고 검색해서 품목들 나오는데...
7,작살나리,2020-02-20 16:39:58-07:00,5,0,역시 실망안시킴 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,거리멀어서 매장못가는게 아쉬운\n제주도거주자 1인...
